# 线性回归的从零开始实现

### 1. 生成数据集

In [1]:
from mxnet import nd
from mxnet import autograd as ag

In [2]:
num_inputs = 2
num_examples = 1001
batch_size = 10
true_w = [2, -3.4]
true_b = 4.2
x = nd.random_normal(0, 1, shape=(num_examples, num_inputs))
y = x[:,0] * true_w[0] + x[:, 1] * true_w[1] + true_b 
y = y + 0.01*nd.random_normal(shape=y.shape)

## 2. 读取数据

In [3]:
import random
def data_loader(data, label, batch_size):
    index = nd.arange(len(data))
    random.shuffle(index)
    for i in range(0, len(index),batch_size):
        j = nd.array(index[i: min(i + batch_size, num_examples)])
        yield data.take(j), label.take(j)

### 3. 初始化模型参数

In [4]:
w = nd.random_normal(shape=(num_inputs,1))
b = nd.zeros(1)
params = [w, b]
for param in params:
    param.attach_grad()

### 4.定义模型

In [5]:
def linreg(x):
    return nd.dot(x, w) + b

### 5.损失函数

In [6]:
def L2_loss(y_hat, y):
    return 0.5 *(y_hat - y.reshape(y_hat.shape))**2

### 6.定义优化算法

In [7]:
def sgd(params, lr, batch_size):
    for param in params:
        param[:] = param - lr /batch_size* param.grad

###  7. 训练模型

In [8]:
num_epochs = 3
lr = 0.01
for epoch in range(num_epochs):
    total_loss = 0.
    for data, label in data_loader(x, y, batch_size):
        with ag.record():
            output = linreg(data)
            loss = L2_loss(output, label)
        loss.backward()
        sgd(params, lr, batch_size)
        total_loss += loss.sum().asscalar()
    print("Epoch %d, Loss: %f" %(epoch, total_loss/num_examples))

Epoch 0, Loss: 4.889596
Epoch 1, Loss: 0.638083
Epoch 2, Loss: 0.076978


In [9]:
params

[
 [[ 1.9625334]
  [-3.3460512]]
 <NDArray 2x1 @cpu(0)>, 
 [3.998853]
 <NDArray 1 @cpu(0)>]

In [10]:
true_w

[2, -3.4]

## 小结

* 可以看出，仅使用 NDArray 和`autograd`就可以很容易地实现一个模型。在接下来的章节中，我们会在此基础上描述更多深度学习模型，并介绍怎样使用更简洁的代码（例如下一节）来实现它们。


## 练习

* 为什么`squared_loss`函数中需要使用`reshape`函数?
- 在ndarray中（10,1）和（10，）表达的不一样，一个是行向量，一个是列向量，如果不用reshape，将会返回（10,10）的矩阵，这不是我们想要的
* 尝试使用不同的学习率，观察损失函数值的下降快慢。
* 如果样本个数不能被批量大小整除，`data_iter`函数的行为会有什么变化？
- data_iter 中去数据时用到 min(i + batch_size, num_examples),如果最后一批不能被整除将用剩下的数据，而不是整个batch_size
- `sgd`梯度下降中 lr之所以要除以batch_size是因为消除batch_size对学习率的影响；如果对$y=w * x$求导，x样本是一千个1，如果batch_size=1000,那么w.grad为1000，batch_size=100,w.grad=100,batch_size=1,w.grad=1,所以说要除以batch_size，消除它对求导的影响。如果不除以batch_size,如果batch_size=10000或者更大，结果可能直接不收敛，需要调很小的lr；虽然平时不会设置这么大的batch_size


## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/743)

# Gluon 版本

In [11]:
from mxnet import gluon

使用和之前一样的数据集

### 读取数据

In [12]:
dataset = gluon.data.ArrayDataset(x, y)
data_iter = gluon.data.DataLoader(dataset, batch_size=batch_size,shuffle=True)

### 定义模型

In [13]:
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
net.initialize()

### 损失函数

In [14]:
L2Loss = gluon.loss.L2Loss()

### 定义优化方法

In [15]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':0.1})

### 训练模型

In [16]:
num_epochs = 5
lr = 0.01
batch_size = 10
for epoch in range(num_epochs):
    total_loss = 0.
    for data, label in data_iter:
        with ag.record():
            output = net(data)
            loss = L2Loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += loss.sum().asscalar()
    print("Epoch %d, Loss: %f" %(epoch, total_loss/num_examples))

Epoch 0, Loss: 0.874796
Epoch 1, Loss: 0.000050
Epoch 2, Loss: 0.000049
Epoch 3, Loss: 0.000049
Epoch 4, Loss: 0.000049


gluon版本比scratch版本效果好的原因可能是，参数初始化时，gluon版可能设置了一个非常小的参数